In [1]:
import os
import numpy as np
import torch
import time
import pickle
import matplotlib.pyplot as plt
from collections import OrderedDict
from torch.autograd import Variable
from pathlib import Path

%matplotlib inline

In [2]:
pix2pixhd_dir = Path('../src/pix2pixHD/')

import sys
sys.path.append(str(pix2pixhd_dir))

%load_ext autoreload
%autoreload 2

In [3]:
from options.train_options import TrainOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

In [4]:
with open('../data/train_opt.pkl', mode='rb') as f:
    opt = pickle.load(f)
    
iter_path = os.path.join(opt.checkpoints_dir, opt.name, 'iter.txt')

In [5]:
opt.lr = 1e-4
opt.loadSize = 480
opt.label_nc = 0
opt.temp_data = True
opt.input_nc = 6
# opt.G_steps = 10
opt.no_flip = True
opt.model = 'pix2pixHD_tempSmooth'
opt.debug = True
opt.name = 'maskGirl'
opt.dataroot='../data/target/maskGirl/'
# opt.lambda_feat = 5.0
# opt.continue_train = True
opt.load_pretrain = '../checkpoints/maskGirl/'

In [6]:
if opt.continue_train:
    try:
        start_epoch, epoch_iter = np.loadtxt(iter_path , delimiter=',', dtype=int)
    except:
        start_epoch, epoch_iter = 1, 0
    print('Resuming from epoch %d at iteration %d' % (start_epoch, epoch_iter))        
else:    
    start_epoch, epoch_iter = 1, 0

if opt.debug:
    opt.display_freq = 1
    opt.print_freq = 1
    opt.niter = 40
    opt.niter_decay = 40
    opt.max_dataset_size = 10
print(opt)

Namespace(batchSize=1, beta1=0.5, checkpoints_dir='../checkpoints/', continue_train=False, data_type=32, dataroot='../data/target/maskGirl/', debug=True, display_freq=1, display_winsize=512, feat_num=3, fineSize=512, fine_size=480, gpu_ids=[0], input_nc=6, instance_feat=False, isTrain=True, label_feat=False, label_nc=0, lambda_feat=10.0, loadSize=480, load_features=False, load_pretrain='../checkpoints/maskGirl/', lr=0.0001, max_dataset_size=10, model='pix2pixHD_tempSmooth', nThreads=2, n_blocks_global=9, n_blocks_local=3, n_clusters=10, n_downsample_E=4, n_downsample_global=4, n_layers_D=3, n_local_enhancers=1, name='maskGirl', ndf=64, nef=16, netG='global', ngf=64, niter=40, niter_decay=40, niter_fix_global=0, no_flip=True, no_ganFeat_loss=False, no_html=False, no_instance=True, no_lsgan=False, no_vgg_loss=False, norm='instance', num_D=2, output_nc=3, phase='train', pool_size=0, print_freq=1, resize_or_crop='scale_width', save_epoch_freq=10, save_latest_freq=640, serial_batches=False,

In [7]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
dataset_size = len(data_loader)
print('#training images = %d' % dataset_size)
    
# start_epoch, epoch_iter = 1, 0
total_steps = (start_epoch-1) * dataset_size + epoch_iter
display_delta = total_steps % opt.display_freq
print_delta = total_steps % opt.print_freq
save_delta = total_steps % opt.save_latest_freq

CustomDatasetDataLoader
dataset [AlignedDataset] was created
#training images = 10


In [8]:
model = create_model(opt)
visualizer = Visualizer(opt)

GlobalGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(6, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (12): ReLU(inplace)
    (13): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (14): InstanceNorm2d(1024, eps=1e-05, momentum=0.1, affine=False, track_runn

create web directory ../checkpoints/maskGirl/web...


In [9]:
for epoch in range(start_epoch, opt.niter + opt.niter_decay + 1):
    epoch_start_time = time.time()
    print(start_epoch)
    if epoch != start_epoch:
        epoch_iter = epoch_iter % dataset_size
    for i, data in enumerate(dataset, start=epoch_iter):
        iter_start_time = time.time()
        total_steps += opt.batchSize
        epoch_iter += opt.batchSize

        # whether to collect output images
        save_fake = total_steps % opt.display_freq == display_delta
        
        ############## Forward Pass ######################
        losses, generated = model(Variable(data['label']), Variable(data['inst']), 
            Variable(data['image']), Variable(data['feat']), infer=save_fake)
        
        # sum per device losses
        losses = [ torch.mean(x) if not isinstance(x, int) else x for x in losses ]
        loss_dict = dict(zip(model.module.loss_names, losses))

        # calculate final loss scalar
        loss_D = (loss_dict['D_fake'] + loss_dict['D_real']) * 0.5
        loss_G = loss_dict['G_GAN'] + loss_dict.get('G_GAN_Feat',0) + \
                 loss_dict.get('G_VGG',0)
        # loss_Dt = (loss_dict['D_fake_temp'] + loss_dict['D_real_temp']) * 0.5
        
        ############### Backward Pass ####################
        # update generator weights
        model.module.optimizer_G.zero_grad()
        loss_G.backward()
        model.module.optimizer_G.step()

        # update discriminator weights
        model.module.optimizer_D.zero_grad()
        loss_D.backward()
        model.module.optimizer_D.step()
        #call(["nvidia-smi", "--format=csv", "--query-gpu=memory.used,memory.free"]) 

        ############## Display results and errors ##########
        ### print out errors
        if total_steps % opt.print_freq == print_delta:
            errors = {k: v.data[0] if not isinstance(v, int) else v for k, v in loss_dict.items()}
            t = (time.time() - iter_start_time) / opt.batchSize
            visualizer.print_current_errors(epoch, epoch_iter, errors, t)
            visualizer.plot_current_errors(errors, total_steps)

        ### display output images
        if save_fake:
            visuals = OrderedDict([('input_label', util.tensor2label(data['label'][0][3:,:,:], opt.label_nc)),
                                   ('synthesized_image', util.tensor2im(generated.data[0])),
                                   ('real_image', util.tensor2im(data['image'][0][3:,:,:]))])
            visualizer.display_current_results(visuals, epoch, total_steps)

        ### save latest model
        if total_steps % opt.save_latest_freq == save_delta:
            print('saving the latest model (epoch %d, total_steps %d)' % (epoch, total_steps))
            model.module.save('latest')            
            np.savetxt(iter_path, (epoch, epoch_iter), delimiter=',', fmt='%d')

        if epoch_iter >= dataset_size:
            break
       
    # end of epoch 
    iter_end_time = time.time()
    print('End of epoch %d / %d \t Time Taken: %d sec' %
          (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))

    ### save model for this epoch
    if epoch % opt.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_steps))        
        model.module.save('latest')
        model.module.save(epoch)
        np.savetxt(iter_path, (epoch+1, 0), delimiter=',', fmt='%d')

    ### instead of only training the local enhancer, train the entire network after certain iterations
    if (opt.niter_fix_global != 0) and (epoch == opt.niter_fix_global):
        model.module.update_fixed_params()

    ### linearly decay learning rate after certain iterations
    if epoch > opt.niter:
        model.module.update_learning_rate()
        
torch.cuda.empty_cache()

1


/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:42: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


(epoch: 1, iters: 1, time: 5.117) G_GAN: 0.898 G_GAN_Feat: 10.730 G_VGG: 12.170 D_real: 0.451 D_fake: 0.359 
(epoch: 1, iters: 2, time: 0.610) G_GAN: 0.736 G_GAN_Feat: 10.675 G_VGG: 12.959 D_real: 0.435 D_fake: 0.424 
(epoch: 1, iters: 3, time: 0.561) G_GAN: 0.698 G_GAN_Feat: 8.792 G_VGG: 12.288 D_real: 0.355 D_fake: 0.426 
(epoch: 1, iters: 4, time: 0.556) G_GAN: 0.670 G_GAN_Feat: 8.574 G_VGG: 12.607 D_real: 0.316 D_fake: 0.420 
(epoch: 1, iters: 5, time: 0.570) G_GAN: 0.624 G_GAN_Feat: 6.756 G_VGG: 11.699 D_real: 0.356 D_fake: 0.437 
(epoch: 1, iters: 6, time: 0.580) G_GAN: 0.496 G_GAN_Feat: 8.854 G_VGG: 11.930 D_real: 0.334 D_fake: 0.546 
(epoch: 1, iters: 7, time: 0.568) G_GAN: 0.666 G_GAN_Feat: 6.964 G_VGG: 11.430 D_real: 0.407 D_fake: 0.407 
(epoch: 1, iters: 8, time: 0.528) G_GAN: 0.659 G_GAN_Feat: 6.887 G_VGG: 11.512 D_real: 0.430 D_fake: 0.399 
(epoch: 1, iters: 9, time: 0.536) G_GAN: 0.596 G_GAN_Feat: 6.150 G_VGG: 11.085 D_real: 0.445 D_fake: 0.446 
(epoch: 1, iters: 10, time

(epoch: 8, iters: 5, time: 0.524) G_GAN: 0.681 G_GAN_Feat: 4.973 G_VGG: 9.262 D_real: 0.380 D_fake: 0.408 
(epoch: 8, iters: 6, time: 0.552) G_GAN: 0.610 G_GAN_Feat: 4.374 G_VGG: 9.182 D_real: 0.684 D_fake: 0.466 
(epoch: 8, iters: 7, time: 0.531) G_GAN: 0.804 G_GAN_Feat: 5.142 G_VGG: 9.482 D_real: 0.367 D_fake: 0.346 
(epoch: 8, iters: 8, time: 0.521) G_GAN: 0.758 G_GAN_Feat: 5.283 G_VGG: 9.462 D_real: 0.327 D_fake: 0.357 
(epoch: 8, iters: 9, time: 0.529) G_GAN: 0.581 G_GAN_Feat: 4.505 G_VGG: 9.218 D_real: 0.357 D_fake: 0.453 
(epoch: 8, iters: 10, time: 0.526) G_GAN: 0.883 G_GAN_Feat: 5.657 G_VGG: 9.738 D_real: 0.598 D_fake: 0.299 
End of epoch 8 / 80 	 Time Taken: 6 sec
1
(epoch: 9, iters: 1, time: 0.521) G_GAN: 0.545 G_GAN_Feat: 3.935 G_VGG: 9.228 D_real: 0.308 D_fake: 0.494 
(epoch: 9, iters: 2, time: 0.542) G_GAN: 0.601 G_GAN_Feat: 4.792 G_VGG: 9.104 D_real: 0.326 D_fake: 0.458 
(epoch: 9, iters: 3, time: 0.554) G_GAN: 0.901 G_GAN_Feat: 5.731 G_VGG: 9.511 D_real: 0.372 D_fake: 0

(epoch: 15, iters: 8, time: 0.527) G_GAN: 1.184 G_GAN_Feat: 7.350 G_VGG: 9.505 D_real: 0.333 D_fake: 0.189 
(epoch: 15, iters: 9, time: 0.528) G_GAN: 0.877 G_GAN_Feat: 5.183 G_VGG: 8.462 D_real: 0.220 D_fake: 0.300 
(epoch: 15, iters: 10, time: 0.516) G_GAN: 0.982 G_GAN_Feat: 5.267 G_VGG: 9.086 D_real: 0.291 D_fake: 0.290 
End of epoch 15 / 80 	 Time Taken: 6 sec
1
(epoch: 16, iters: 1, time: 0.553) G_GAN: 1.062 G_GAN_Feat: 6.237 G_VGG: 9.287 D_real: 0.222 D_fake: 0.225 
(epoch: 16, iters: 2, time: 0.533) G_GAN: 0.873 G_GAN_Feat: 6.397 G_VGG: 9.479 D_real: 0.277 D_fake: 0.292 
(epoch: 16, iters: 3, time: 0.512) G_GAN: 1.044 G_GAN_Feat: 6.440 G_VGG: 9.325 D_real: 0.291 D_fake: 0.202 
(epoch: 16, iters: 4, time: 0.540) G_GAN: 1.007 G_GAN_Feat: 5.828 G_VGG: 8.899 D_real: 0.187 D_fake: 0.233 
(epoch: 16, iters: 5, time: 0.534) G_GAN: 1.051 G_GAN_Feat: 6.912 G_VGG: 9.217 D_real: 0.588 D_fake: 0.219 
(epoch: 16, iters: 6, time: 0.533) G_GAN: 0.702 G_GAN_Feat: 4.866 G_VGG: 8.736 D_real: 0.336

...

(epoch: 69, iters: 1, time: 0.549) G_GAN: 1.439 G_GAN_Feat: 7.254 G_VGG: 8.214 D_real: 0.121 D_fake: 0.085 
(epoch: 69, iters: 2, time: 0.566) G_GAN: 1.116 G_GAN_Feat: 6.988 G_VGG: 8.147 D_real: 0.036 D_fake: 0.171 
(epoch: 69, iters: 3, time: 0.560) G_GAN: 1.961 G_GAN_Feat: 6.344 G_VGG: 8.105 D_real: 0.412 D_fake: 0.037 
(epoch: 69, iters: 4, time: 0.554) G_GAN: 1.512 G_GAN_Feat: 7.549 G_VGG: 7.771 D_real: 0.042 D_fake: 0.106 
(epoch: 69, iters: 5, time: 0.573) G_GAN: 0.976 G_GAN_Feat: 6.564 G_VGG: 7.203 D_real: 0.081 D_fake: 0.218 
(epoch: 69, iters: 6, time: 0.559) G_GAN: 1.704 G_GAN_Feat: 7.897 G_VGG: 8.227 D_real: 0.156 D_fake: 0.065 
(epoch: 69, iters: 7, time: 0.557) G_GAN: 0.767 G_GAN_Feat: 6.551 G_VGG: 7.797 D_real: 0.078 D_fake: 0.386 
(epoch: 69, iters: 8, time: 0.570) G_GAN: 2.244 G_GAN_Feat: 7.579 G_VGG: 7.991 D_real: 0.129 D_fake: 0.047 
(epoch: 69, iters: 9, time: 0.559) G_GAN: 1.982 G_GAN_Feat: 7.936 G_VGG: 8.283 D_real: 0.035 D_fake: 0.037 
(epoch: 69, iters: 10, time:

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fc39c7a5f60>>
Traceback (most recent call last):
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/pytho

(epoch: 70, iters: 1, time: 0.553) G_GAN: 1.986 G_GAN_Feat: 6.120 G_VGG: 7.110 D_real: 0.155 D_fake: 0.031 
(epoch: 70, iters: 2, time: 0.553) G_GAN: 1.839 G_GAN_Feat: 6.427 G_VGG: 7.441 D_real: 0.130 D_fake: 0.042 
(epoch: 70, iters: 3, time: 0.551) G_GAN: 1.861 G_GAN_Feat: 7.647 G_VGG: 7.704 D_real: 0.080 D_fake: 0.039 
(epoch: 70, iters: 4, time: 0.553) G_GAN: 1.275 G_GAN_Feat: 7.210 G_VGG: 7.767 D_real: 0.044 D_fake: 0.124 
(epoch: 70, iters: 5, time: 0.561) G_GAN: 0.607 G_GAN_Feat: 6.882 G_VGG: 8.738 D_real: 0.068 D_fake: 0.518 
(epoch: 70, iters: 6, time: 0.559) G_GAN: 1.812 G_GAN_Feat: 7.873 G_VGG: 8.544 D_real: 0.051 D_fake: 0.036 
(epoch: 70, iters: 7, time: 0.557) G_GAN: 1.607 G_GAN_Feat: 6.632 G_VGG: 8.121 D_real: 0.361 D_fake: 0.064 
(epoch: 70, iters: 8, time: 0.561) G_GAN: 1.628 G_GAN_Feat: 6.434 G_VGG: 8.013 D_real: 0.066 D_fake: 0.071 
(epoch: 70, iters: 9, time: 0.557) G_GAN: 1.375 G_GAN_Feat: 6.136 G_VGG: 7.717 D_real: 0.116 D_fake: 0.113 
(epoch: 70, iters: 10, time:

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fc39c5dd400>>
Traceback (most recent call last):
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/yaosy/anaconda3/e

(epoch: 75, iters: 1, time: 0.542) G_GAN: 1.539 G_GAN_Feat: 6.360 G_VGG: 6.883 D_real: 0.034 D_fake: 0.101 
(epoch: 75, iters: 2, time: 0.547) G_GAN: 1.497 G_GAN_Feat: 6.638 G_VGG: 7.471 D_real: 0.037 D_fake: 0.079 
(epoch: 75, iters: 3, time: 0.548) G_GAN: 0.885 G_GAN_Feat: 6.116 G_VGG: 7.363 D_real: 0.034 D_fake: 0.266 
(epoch: 75, iters: 4, time: 0.554) G_GAN: 1.804 G_GAN_Feat: 8.502 G_VGG: 9.213 D_real: 0.106 D_fake: 0.045 
(epoch: 75, iters: 5, time: 0.564) G_GAN: 1.833 G_GAN_Feat: 7.546 G_VGG: 8.170 D_real: 0.062 D_fake: 0.047 
(epoch: 75, iters: 6, time: 0.552) G_GAN: 1.914 G_GAN_Feat: 6.792 G_VGG: 7.823 D_real: 0.189 D_fake: 0.031 
(epoch: 75, iters: 7, time: 0.548) G_GAN: 1.805 G_GAN_Feat: 7.361 G_VGG: 7.905 D_real: 0.066 D_fake: 0.038 
(epoch: 75, iters: 8, time: 0.558) G_GAN: 1.824 G_GAN_Feat: 7.959 G_VGG: 8.033 D_real: 0.037 D_fake: 0.047 
(epoch: 75, iters: 9, time: 0.555) G_GAN: 1.907 G_GAN_Feat: 7.380 G_VGG: 7.020 D_real: 0.086 D_fake: 0.022 
(epoch: 75, iters: 10, time: